# Day 4 - Lab 1: Automated Testing & Quality Assurance (Solution)

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Introduction:**
This solution notebook provides the complete prompts and explanations for generating a robust test suite. It covers generating simple tests, refactoring them into more advanced patterns, and creating the necessary fixtures for professional-grade database testing.

## Step 1: Setup

**Explanation:**
We load the application's source code to provide the LLM with the necessary context to write accurate tests. A good prompt for test generation should always include the code that needs to be tested.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from utils import setup_llm_client, get_completion, save_artifact, load_artifact

client, model_name, api_provider = setup_llm_client()

# In a real scenario, you would load all relevant application files.
# For this solution, we assume the context is available.
try:
    # Assuming the app is structured with main.py, crud.py, models.py, schemas.py
    # A more robust script would concatenate these files.
    main_code = load_artifact("app/main.py") # Simplified for demonstration
    full_app_context = f"""# main.py\n{main_code}\n"""
    print("Successfully loaded application code context.")
except (FileNotFoundError, TypeError):
    full_app_context = ""
    print("Warning: Could not load application code. Lab may not function correctly.")

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Explanation:**
This prompt asks for the most straightforward type of test: one that verifies the application works as expected when given valid input. We specifically ask for tests for the `POST` and `GET` endpoints. The prompt includes the full application code as context, which is crucial for the LLM to understand the API's structure, expected payloads, and responses.

In [ ]:
happy_path_tests_prompt = f"""
You are a Senior QA Engineer writing tests for a FastAPI application using pytest.

Based on the application code provided below, please generate two 'happy path' test functions:
1. A test named `test_create_user` for the `POST /users/` endpoint. It should create a user and assert that the status code is 200 and the response email matches the input.
2. A test named `test_read_users` for the `GET /users/` endpoint. It should assert the status code is 200 and that the response is a list.

**Application Code Context:**
```python
{full_app_context}
```

Your response should be only the raw Python code for the tests, including necessary imports like `TestClient`.
"""

print("--- Generating Happy Path Tests ---")
if full_app_context:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    print(generated_happy_path_tests)
else:
    print("Skipping test generation because app context is missing.")

### Challenge 2 (Intermediate): Parameterizing Edge Case Tests

**Explanation:**
This is a two-step process that teaches a valuable refactoring pattern. First, we would generate individual tests for different error conditions. Then, we provide those generated tests back to the LLM and ask it to refactor them. The key instruction is to use `@pytest.mark.parametrize`, which allows a single test function to be run with multiple different inputs. This results in code that is much cleaner, more concise, and easier to maintain.

In [ ]:
# This prompt assumes we have already generated two separate tests for invalid data.
# For this solution, we combine the request into one.
parametrize_refactor_prompt = f"""
You are a Senior Python Developer who champions writing clean, maintainable test code.

Based on the FastAPI application code below, I need a test for invalid user creation payloads. Instead of writing multiple separate tests, please generate a single, parameterized test function using `@pytest.mark.parametrize`.

The test should be named `test_create_user_invalid_payload` and should check at least two scenarios:
1. A payload with an invalid email address.
2. A payload with a password that is too short (e.g., less than 6 characters).

In both scenarios, the test must assert that the API returns a `422 Unprocessable Entity` status code.

**Application Code Context:**
```python
{full_app_context}
```

Output only the raw Python code for the single parameterized test function.
"""

print("--- Generating Parameterized Edge Case Test ---")
if full_app_context:
    generated_parameterized_test = get_completion(parametrize_refactor_prompt, client, model_name, api_provider)
    print(generated_parameterized_test)
else:
    print("Skipping test generation because app context is missing.")

### Challenge 3 (Advanced): Testing with an Isolated Database

**Explanation:**
This is the most advanced and most important concept in this lab. Running tests against your real development database is risky and can lead to flaky tests. A professional testing setup uses an isolated, temporary database for each test run.

This prompt asks the LLM to generate a `pytest` fixture. A fixture is a function that runs before each test. This particular fixture does the following:
1.  Sets up an in-memory SQLite database.
2.  Creates all the necessary tables.
3.  Uses a FastAPI feature to override the `get_db` dependency, so that during the test, API endpoints connect to the temporary database instead of the real one.
4.  Yields the database session to the test function.
5.  After the test is finished, it cleans up and closes the connection.

Generating this complex fixture saves a tremendous amount of time and boilerplate coding.

In [ ]:
db_fixture_prompt = f"""
You are an expert in Python testing with pytest and FastAPI.

I need to create a `pytest` fixture to provide an isolated, in-memory SQLite database session for each test run. This is a critical best practice for testing database-connected applications.

Please generate the Python code for a file named `tests/conftest.py` that contains this fixture.

The fixture should:
1. Be named `test_db`.
2. Configure a SQLAlchemy engine for an in-memory SQLite database.
3. Create all database tables before the test session starts.
4. Override the `get_db` dependency from the main FastAPI app to yield a session from this test database.
5. Clean up the database tables after the test session is complete.

**Application Code Context:**
```python
{full_app_context}
```

Output only the raw Python code for the `conftest.py` file.
"""

print("--- Generating Pytest DB Fixture ---")
if full_app_context:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    print(generated_db_fixture)
    # In the lab, students would now create 'tests/conftest.py' and 'tests/test_main.py'
    # and assemble all the generated test code into the final test suite.
    # save_artifact(generated_db_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases, refactor tests for maintainability, and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality. In the next lab, we will use this test suite to help us debug and improve our code.